In [ ]:
pip install openai==0.28


In [ ]:
pip install --upgrade openai


In [ ]:
import openai
import requests
import os

# Set your OpenAI API Key
openai.api_key = "your_openai_api_key"

def generate_images(prompt, size="1024x1024", num_images=1, save_images=True):
    """
    Generate images from a text prompt using OpenAI's DALL-E API.

    Args:
        prompt (str): The text description of the image.
        size (str): The size of the generated images. Options: '256x256', '512x512', '1024x1024'.
        num_images (int): The number of images to generate.
        save_images (bool): Whether to save the images locally.

    Returns:
        list: List of image URLs or error messages.
    """
    try:
        print(f"Generating {num_images} image(s) for prompt: '{prompt}' with size {size}")

        # Call the OpenAI API for image generation
        response = openai.Image.create(
            prompt=prompt,
            n=num_images,  # Number of images to generate
            size=size,
            response_format="url"  # Response format for the image URL
        )

        image_urls = [data['url'] for data in response['data']]
        print("Images successfully generated!")

        if save_images:
            save_generated_images(image_urls, prompt)

        return image_urls

    except Exception as e:
        print(f"Error: {e}")
        log_error(e)
        return [f"Error: {e}"]

def save_generated_images(image_urls, prompt):
    """
    Save the generated images locally using the URLs.

    Args:
        image_urls (list): List of image URLs.
        prompt (str): The prompt used to generate the images (used for naming files).
    """
    output_dir = "generated_images"
    os.makedirs(output_dir, exist_ok=True)

    for i, url in enumerate(image_urls):
        try:
            response = requests.get(url)
            response.raise_for_status()
            file_name = f"{output_dir}/{prompt.replace(' ', '_')}_{i + 1}.png"
            with open(file_name, 'wb') as file:
                file.write(response.content)
            print(f"Image saved as {file_name}")
        except Exception as e:
            print(f"Failed to save image {i + 1}: {e}")

def log_error(error):
    """
    Log errors to a file.

    Args:
        error (Exception): The error to log.
    """
    with open("error_log.txt", "a") as log_file:
        log_file.write(f"{str(error)}\n")

if __name__ == "__main__":
    # User interaction for prompt input
    print("Welcome to the Text-to-Image Generator!")
    prompt = input("Enter the prompt for the image: ")
    size = input("Enter the image size (256x256, 512x512, 1024x1024): ") or "1024x1024"
    num_images = int(input("Enter the number of images to generate: ") or 1)

    # Generate images
    image_urls = generate_images(prompt, size=size, num_images=num_images)

    # Display the generated image URLs
    print("\nGenerated Image URLs:")
    for url in image_urls:
        print(url)
